In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time
import csv
import queue

In [ ]:
model_path = "/Users/beni/Desktop/Climbing Technique Tracker/Climbing_Tracker/pose_landmarker_full.task"

In [ ]:
"""The code in this block is partly taken from the official MediaPipe documentation and can be found 
here: https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker/python#live-stream_1"""

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
PoseLandmarkerResult = mp.tasks.vision.PoseLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

saved_results = queue.Queue()


def print_result(result: PoseLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    print('pose landmarker result: {}'.format(result))
    saved_results.put(result)


options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM, #sets the running mode to live stream
    result_callback=print_result)


"""The following code is written by myself using the OpenCV and MediaPipe documentation for help which
can be found in the readme file"""

#check how many frames per second Qsense sensors output
frames_per_second = 10
time_between_frames = 1 / frames_per_second


with PoseLandmarker.create_from_options(options) as landmarker:
    #Use openCV VideoCapture to start capturing from the webcam
    start_time = time.time()

    capture = cv2.VideoCapture(0)

    #Create a loop to read the latest frame from the camera using VideoCaptureRead()
    while capture.isOpened():

        ret, frame = capture.read()
     
        if not ret:
            print("broken")
            break

        #Convert the frame receieved from OpenCV to a MediaPipe Image Object
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
      
        landmarker.detect_async(mp_image, timestamp_ms = int(time.time() * 1000))
        
        #color = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imshow('frame', frame) #first param is window name
        cv2.waitKey(10)

        """The following frame limiter was taken from the following online answer:
         https://www.quora.com/How-do-I-decrease-the-frames-per-second-in-OpenCV-python"""
        
        elapsed_time = time.time() - start_time 
        time_to_wait = time_between_frames - elapsed_time 
        if time_to_wait > 0: 
            time.sleep(time_to_wait)git
    

In [ ]:
results = []
intermediate_array = []

while not saved_results.empty():
    r = saved_results.get()
    intermediate_array.append(r)
    results.append(intermediate_array)


print("Collected", len(results), "results")
print(results if results else "No results found")




with open('results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)